In [1]:
# demonstrate a simple computation graph

import torch
import torch.nn.functional as F
import torch

y = torch.tensor([1.0])
x1 = torch.tensor([1.1])
w1 = torch.tensor([2.2], requires_grad=True)
b = torch.tensor([3.3], requires_grad=True)
z = w1 * x1 + b
a = torch.sigmoid(z)
loss = F.binary_cross_entropy(a, y)

grad_L_w1 = torch.autograd.grad(loss, w1, retain_graph=True)
grad_L_b = torch.autograd.grad(loss, b, retain_graph=True)
print(f'grad_L_w1: {grad_L_w1}')
print(f'grad_L_b: {grad_L_b}')

# all the gradients can be autocomputed using the backward method
loss.backward()
print(f'w1.grad: {w1.grad}')
print(f'b.grad: {b.grad}')

c:\Users\sumukherjee\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\_subclasses\functional_tensor.py:275: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_numpy.cpp:81.)
  cpu = _conversion_method_template(device=torch.device("cpu"))


grad_L_w1: (tensor([-0.0036]),)
grad_L_b: (tensor([-0.0033]),)
w1.grad: tensor([-0.0036])
b.grad: tensor([-0.0033])


In [2]:
class NeuralNetwork(torch.nn.Module):
    def __init__(self, num_inputs, num_outputs):
        super().__init__()

        self.layers = torch.nn.Sequential(
            
            #1st hidden layer
            torch.nn.Linear(num_inputs, 30),
            torch.nn.ReLU(),

            # 2nd hidden layer
            torch.nn.Linear(30, 20),
            torch.nn.ReLU(),

            # output layer
            torch.nn.Linear(20, num_outputs)
        )

    def forward(self, x):
        logits = self.layers(x)
        return logits
    
model = NeuralNetwork(50, 3)
print(model)

# How many trainable parameters does the model have?
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'{total_params} total parameters.')

print(model.layers[0].weight)
print(model.layers[0].weight.shape)

torch.manual_seed(123)
X = torch.rand(1, 50)

with torch.no_grad():
    out = torch.softmax(model(X), dim=1)

print(out)


NeuralNetwork(
  (layers): Sequential(
    (0): Linear(in_features=50, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=3, bias=True)
  )
)
2213 total parameters.
Parameter containing:
tensor([[-0.1280, -0.0568, -0.0601,  ..., -0.1211,  0.1086, -0.0745],
        [ 0.1082,  0.1265,  0.1115,  ...,  0.0241,  0.0171,  0.0766],
        [ 0.0459, -0.0462,  0.1265,  ..., -0.1226, -0.0136, -0.0999],
        ...,
        [-0.0170, -0.1334, -0.0948,  ..., -0.1075,  0.0225,  0.1199],
        [ 0.0777, -0.0547,  0.0644,  ...,  0.1240, -0.0051, -0.0009],
        [ 0.0809,  0.1244, -0.0679,  ...,  0.0335,  0.0099, -0.0165]],
       requires_grad=True)
torch.Size([30, 50])
tensor([[0.3763, 0.2828, 0.3409]])


In [3]:
# Understanding data loaders

X_train = torch.tensor(
    [
        [-1.2, 3.1],
        [-0.9, 2.9],
        [-0.5, 2.6],
        [2.3, 1.1],
        [2.7, -1.5]
    ]
)

y_train = torch.tensor([0, 0, 0, 1, 1])

X_test = torch.tensor(
    [
        [-1.1, 3.0],
        [2.4, 1.0]
    ]
)

y_test = torch.tensor([0, 1])

from torch.utils.data import Dataset

class ToyDataset(Dataset):
    def __init__(self, X, y):
        self.features = X
        self.label = y

    def __getitem__(self, index):
        return self.features[index], self.label[index]
    
    def __len__(self):
        return self.label.shape[0]
    
train_dataset = ToyDataset(X_train, y_train)
test_dataset = ToyDataset(X_test, y_test)

# Instantiate a data loader
from torch.utils.data import DataLoader

train_loader = DataLoader(
    dataset = train_dataset,
    batch_size = 2,
    shuffle=True,
    num_workers=0
)

test_loader = DataLoader(
    dataset = test_dataset,
    batch_size = 2,
    shuffle=False,
    num_workers=0
)

# iterate over the training dataloader
for idx, (x,y) in enumerate(train_loader):
    print(f'batch {idx + 1}:')
    print(x)
    print(y)

batch 1:
tensor([[ 2.3000,  1.1000],
        [-0.5000,  2.6000]])
tensor([1, 0])
batch 2:
tensor([[-1.2000,  3.1000],
        [-0.9000,  2.9000]])
tensor([0, 0])
batch 3:
tensor([[ 2.7000, -1.5000]])
tensor([1])


In [4]:
def compute_accuracy(model, data_loader):
    
    model = model.eval()
    correct = 0.0
    total_examples = 0

    for idx, (features, labels) in enumerate(data_loader):
        with torch.no_grad():
            logits = model(features)
            predictions = torch.argmax(logits, dim=1)
            correct += torch.sum((predictions == labels).float())
            total_examples += labels.shape[0]

    return (correct / total_examples).item()

In [5]:
# Train the neural network

import torch.nn.functional as F

torch.manual_seed(123)
model = NeuralNetwork(num_inputs=2, num_outputs=2)

# Count number of trainable parameters
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'{total_params} total parameters.')

optimizer = torch.optim.SGD(model.parameters(), lr=0.5)

num_epochs = 5

for epoch in range(num_epochs):
    model.train()

    for batch_idx, (features, labels) in enumerate(train_loader):
        logits = model(features)

        loss = F.cross_entropy(logits, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

         ### LOGGING
        print(f"Epoch: {epoch+1:03d}/{num_epochs:03d}"
        f" | Batch {batch_idx:03d}/{len(train_loader):03d}"
        f" | Train Loss: {loss:.2f}")

model.eval()

with torch.no_grad():
    outputs = model(X_train)

print(outputs)

torch.set_printoptions(sci_mode=False)
probas = torch.softmax(outputs, dim=1)
print(probas)

predictions = torch.argmax(probas, dim=1)
print(predictions)

# How many did we get right in the training set?
predictions == y_train
torch.sum(predictions == y_train)

print(compute_accuracy(model, train_loader))
print(compute_accuracy(model, test_loader))

752 total parameters.
Epoch: 001/005 | Batch 000/003 | Train Loss: 0.75
Epoch: 001/005 | Batch 001/003 | Train Loss: 0.90
Epoch: 001/005 | Batch 002/003 | Train Loss: 0.65
Epoch: 002/005 | Batch 000/003 | Train Loss: 0.28
Epoch: 002/005 | Batch 001/003 | Train Loss: 0.92
Epoch: 002/005 | Batch 002/003 | Train Loss: 0.00
Epoch: 003/005 | Batch 000/003 | Train Loss: 0.04
Epoch: 003/005 | Batch 001/003 | Train Loss: 0.00
Epoch: 003/005 | Batch 002/003 | Train Loss: 0.32
Epoch: 004/005 | Batch 000/003 | Train Loss: 0.00
Epoch: 004/005 | Batch 001/003 | Train Loss: 0.01
Epoch: 004/005 | Batch 002/003 | Train Loss: 0.00
Epoch: 005/005 | Batch 000/003 | Train Loss: 0.00
Epoch: 005/005 | Batch 001/003 | Train Loss: 0.01
Epoch: 005/005 | Batch 002/003 | Train Loss: 0.00
tensor([[ 3.8888, -4.6334],
        [ 3.3716, -4.0287],
        [ 2.6617, -3.1935],
        [-2.3332,  2.2271],
        [-3.5767,  3.3827]])
tensor([[    0.9998,     0.0002],
        [    0.9994,     0.0006],
        [    0.9971

In [9]:
# PyTorch for GPU devices

print(torch.xpu.is_available())

tensor_size = 20000

test_tensor_1 = torch.rand(tensor_size, tensor_size)
test_tensor_2 = torch.rand(tensor_size, tensor_size)

xpu_tensor_1 = test_tensor_1.to('xpu')
xpu_tensor_2 = test_tensor_2.to('xpu')

# perform matrix multiplication and measure the time
import time
start_time = time.time()
cpu_product = test_tensor_1.matmul(test_tensor_2)
end_time = time.time()
print(cpu_product)
print(f'CPU time: {end_time - start_time}')

del test_tensor_1
del test_tensor_2
del cpu_product

start_time = time.time()
xpu_product = xpu_tensor_1.matmul(xpu_tensor_2)
end_time = time.time()
print(xpu_product)
print(f'XPU time: {end_time - start_time}')

del xpu_tensor_1
del xpu_tensor_2
del xpu_product


True
tensor([[4995.6992, 5045.0278, 5056.1577,  ..., 5051.9419, 5086.8506,
         5043.0430],
        [4983.2793, 4989.6489, 5014.4165,  ..., 5040.7515, 5071.5649,
         5029.7056],
        [4975.0132, 4981.0435, 4979.8691,  ..., 4995.7822, 5037.0308,
         5002.6919],
        ...,
        [4988.9795, 4991.8955, 5014.2637,  ..., 5047.7988, 5060.3491,
         5026.4438],
        [4954.4888, 5001.1348, 5029.0981,  ..., 5028.0557, 5051.3140,
         5013.1890],
        [4983.6909, 5001.6387, 5030.7686,  ..., 5030.8921, 5051.7598,
         5028.6831]])
CPU time: 58.75623321533203
tensor([[4995.6919, 5045.0210, 5056.1768,  ..., 5051.9399, 5086.8555,
         5043.0405],
        [4983.2705, 4989.6406, 5014.4072,  ..., 5040.7637, 5071.5625,
         5029.7007],
        [4975.0239, 4981.0474, 4979.8521,  ..., 4995.7812, 5037.0278,
         5002.6816],
        ...,
        [4988.9780, 4991.8945, 5014.2568,  ..., 5047.8008, 5060.3374,
         5026.4536],
        [4954.4800, 5001.1270,

In [7]:
# clear variables from memory


